In [64]:
"""
quick_ab_complete.py  –  BIG KINDS 검색 파라미터 A/B 실험통 합본
실험군
  ▸ baseline   : _score DESC  (기본)
  ▸ date_sort  : 날짜 DESC → _score DESC
  ▸ exact_match: "엔비디아" AND "삼성" AND 반도체
  ▸ mm2        : 3 키워드 중 최소 2개 포함
출력
  * Top-10 기사: 발행일 / 키워드 빈도
  * Precision@10  (제목에 세 키워드 전부 들어간 비율)
"""

import os, re, requests
from collections import Counter
from dotenv import load_dotenv

# ─────────────────────────── 환경 설정 ────────────────────────────
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
assert API_KEY, ".env에 BIGKINDS_KEY 못 찾음 ‥ 각성하세요."

URL = "https://tools.kinds.or.kr/search/news"
DAY_FROM, DAY_UNTIL = "2025-06-18", "2025-06-20"   # UNTIL은 해당 일자 '제외'
KW = ["엔비디아", "삼성", "반도체"]                # 키워드 리스트 (빈도 계산용)

# ─────────────────────────── 헬퍼 함수 ────────────────────────────
def make_payload(query: str, sort, hilight: int = 200) -> dict:
    """빅카인즈 POST용 JSON payload 생성"""
    return {
        "access_key": API_KEY,
        "argument": {
            "query": query,
            "published_at": {"from": DAY_FROM, "until": DAY_UNTIL},
            "provider": [], "category": [],
            "category_incident": [], "provider_subject": [], "subject_info": [],
            "sort": sort,
            "hilight": hilight,
            "return_from": 0,
            "return_size": 200,
            # 필드 목록: API 매뉴얼 기준
            "fields": [
                "title", "news_id", "published_at", "content",
                "provider", "byline", "provider_link_page",
                "dateline", "enveloped_at", "hilight"
            ]
        }
    }

def kw_freq(text: str) -> Counter:
    """본문 + 제목에서 키워드 등장 횟수 세서 Counter 반환"""
    cnt = Counter()
    for kw in KW:
        cnt[kw] = len(re.findall(kw, text))
    return cnt

def run_exp(name: str, payload: dict) -> None:
    """실험 한 번 실행하고 결과 요약 출력"""
    print(f"\n### [{name}]")
    resp = requests.post(URL, json=payload, timeout=15)
    resp.raise_for_status()

    data = resp.json()["return_object"]
    docs = data["documents"][:10]  # 상위 10개만
    total_hits = data["total_hits"]
    print(f"total_hits = {total_hits:,}")

    prec10 = 0
    for idx, d in enumerate(docs, 1):
        title = d["title"]
        counts = kw_freq(d["title"] + d["content"])
        if all(counts[k] > 0 for k in KW):
            prec10 += 1

        published = d["published_at"][:10]
        # _score는 응답에 포함되지 않으므로 대신 N/A 출력
        print(f"{idx:>2}. {published}   "
              f"{' / '.join(f'{k}:{counts[k]}' for k in KW)}   "
              f"{title[:70]}")

    print(f"Precision@10 (제목 3키워드 포함) = {prec10}/10 = {prec10/10:.2f}")

# ─────────────────────────── 실험 세트 정의 ─────────────────────────
EXPERIMENTS = {
    "baseline":    make_payload(
        "엔비디아 삼성 반도체",
        {"_score": "desc"}
    ),
    "date_sort":   make_payload(
        "엔비디아 삼성 반도체",
        [{"date": "desc"}, {"_score": "desc"}]
    ),
    "exact_match": make_payload(
        '"엔비디아" AND "삼성" AND 반도체',
        {"_score": "desc"}
    ),
    "mm2":         make_payload(
        # 2-of-3 조건: 세 단어 모두 쓰고, OR 절로 최소 2개 이상 포함 유도
        "(엔비디아 삼성 반도체) AND (엔비디아 OR 삼성 OR 반도체)",
        {"_score": "desc"}
    ),
}

# ─────────────────────────── 메인 루프 ────────────────────────────
if __name__ == "__main__":
    for name, payload in EXPERIMENTS.items():
        try:
            run_exp(name, payload)
        except requests.HTTPError as e:
            print(f"⚠️  {name} 실패: {e.response.status_code} {e.response.text[:120]}...")



### [baseline]
total_hits = 31
 1. 2025-06-19   엔비디아:1 / 삼성:3 / 반도체:0   엔비디아가 외면한 '삼성 HBM', AMD·브로드컴은 왜 선택했나
 2. 2025-06-18   엔비디아:2 / 삼성:3 / 반도체:0   삼성, 브로드컴에 HBM3E 8단 공급…'초격차' 회복 발판 마련 [biz-플러스]
 3. 2025-06-19   엔비디아:0 / 삼성:0 / 반도체:4   텍사스인스트루먼트, 美서 반도체 생산 확대…82조원 투자
 4. 2025-06-18   엔비디아:1 / 삼성:1 / 반도체:5   [반도체위클리] AMD AI 칩에 삼성 HBM 탑재…마이크론, HBM4 샘플 공급
 5. 2025-06-18   엔비디아:1 / 삼성:1 / 반도체:4   [브릿지 신간] 박준영 <한국 반도체의 미래 3년>
 6. 2025-06-18   엔비디아:0 / 삼성:3 / 반도체:1   삼성전자 2분기 실적 ‘주춤’⋯ 결국 키는 ‘HBM4 양산’
 7. 2025-06-18   엔비디아:0 / 삼성:3 / 반도체:1   긴장감 흐른 삼성 전략회의…폴더블폰·HBM 사활 건다
 8. 2025-06-18   엔비디아:0 / 삼성:3 / 반도체:0   하반기 생존 전략 위해 머리 맞댄 삼성
 9. 2025-06-18   엔비디아:0 / 삼성:2 / 반도체:3   AI·반도체 대장주 엇갈린 행보…SK는 '조정' 삼성 '매수 전환'
10. 2025-06-18   엔비디아:0 / 삼성:3 / 반도체:1   美서 첫주 판매 신기록 '닌텐도 스위치2', 삼성 파운드리에 단비
Precision@10 (제목 3키워드 포함) = 2/10 = 0.20

### [date_sort]
total_hits = 31
 1. 2025-06-19   엔비디아:1 / 삼성:3 / 반도체:0   엔비디아가 외면한 '삼성 HBM', AMD·브로드컴은 왜 선택했나
 2. 2025-06-19   엔비디아:0 / 삼성:0 / 반도체:4   텍사스인스트루먼트, 美서 

# 빅카인 api 전체 항목 코드 구현

In [63]:
import os, re, requests, textwrap
from collections import Counter
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
assert API_KEY, ".env에 BIGKINDS_KEY 빠져 있음."

URL = "https://tools.kinds.or.kr/search/news"
DAY_FROM, DAY_UNTIL = "2025-06-18", "2025-06-20"
KW = ["엔비디아", "삼성", "반도체"]

def make_payload(query: str, sort, hilight: int = 200) -> dict:
    return {
        "access_key": API_KEY,
        "argument": {
            "query": query,
            "published_at": {"from": DAY_FROM, "until": DAY_UNTIL},
            "provider": [], "category": [],
            "category_incident": [], "provider_subject": [], "subject_info": [],
            "sort": sort,
            "hilight": hilight,
            "return_from": 0,
            "return_size": 200,
            "fields": [
                "title","news_id","published_at","content",
                "provider","byline","provider_link_page",
                "dateline","enveloped_at","hilight","tms_raw_stream"
            ]
        }
    }

def kw_freq(text: str):
    from collections import Counter
    c = Counter()
    for kw in KW:
        c[kw] = len(re.findall(kw, text))
    return c

def run_exp(name, payload):
    print(f"\n### [{name}]")
    r = requests.post(URL, json=payload, timeout=15)
    r.raise_for_status()

    data = r.json()["return_object"]
    docs = data["documents"][:10]
    print(f"total_hits = {data['total_hits']:,}")

    prec10 = 0
    for i, d in enumerate(docs, 1):
        title = d["title"]
        counts = kw_freq(d["title"] + d["content"])
        if all(counts[k] > 0 for k in KW):
            prec10 += 1

        published = d["published_at"][:10]
        url = d.get("provider_link_page", "링크 없음")
        highlight = d.get("hilight", "")
        tms_snip = d.get("tms_raw_stream", "")[:120].replace("\n", " ")
        content_snip = d["content"][:120]

        print(f"{i:>2}. {published}  "
              f"{' / '.join(f'{k}:{counts[k]}' for k in KW)}")
        print(f"    {title}")
        print(f"    🔗 {url}")
        if highlight:
            print("    highlight:", textwrap.shorten(highlight, 120, placeholder='...'))
        if content_snip:
            print("    content  :", textwrap.shorten(content_snip, 120, placeholder='...'))
        if tms_snip:
            print("    tms      :", textwrap.shorten(tms_snip, 120, placeholder='...'))
        print()

    print(f"Precision@10 (제목 3키워드 포함) = {prec10}/10 = {prec10/10:.2f}")

EXPERIMENTS = {
    "baseline": make_payload("엔비디아 삼성 반도체", {"_score": "desc"}),
    "date_sort": make_payload("엔비디아 삼성 반도체",
                              [{"date": "desc"}, {"_score": "desc"}]),
    "exact_match": make_payload('"엔비디아" AND "삼성" AND 반도체', {"_score": "desc"}),
    "mm2": make_payload("(엔비디아 삼성 반도체) AND (엔비디아 OR 삼성 OR 반도체)",
                        {"_score": "desc"}),
}

if __name__ == "__main__":
    for label, pl in EXPERIMENTS.items():
        try:
            run_exp(label, pl)
        except requests.HTTPError as e:
            print(f"⚠️  {label} 실패: {e.response.status_code} {e.response.text[:120]}...")



### [baseline]
total_hits = 29
 1. 2025-06-18  엔비디아:2 / 삼성:3 / 반도체:0
    삼성, 브로드컴에 HBM3E 8단 공급…'초격차' 회복 발판 마련 [biz-플러스]
    🔗 http://www.sedaily.com/NewsView/2GU4DQZVA5
    highlight: ..<b>엔비디아</b>는 세계 AI 칩 시장에서 70% .. <b>삼성</b>전자는 이달 말까지 최신 제품인 HBM3E 12단을 <b>엔비디아</b>에 공급하는 것을 목표로 잡았다...“<b>엔비디아</b>가...
    content  : 삼성전자(005930)가 AMD에 이어 브로드컴에도 5세대 고대역폭메모리(HBM)를 공급한다. 삼성전자는 엔비디아 HBM 공급에서 SK하이닉스(000660)에 밀리며 올 1분기 세계 D램 매출 1위 자리마저 내줬지만
    tms      : 삼성 브로드컴 공급 HBM3E 초격차 회복 발판 마련 biz 플러스 기업 세계 팹리스 시장 삼성전자 AMD 브로드컴 고대 역폭 메모리 HBM 공급 삼성전자 공급 엔비디아 HBM SK하이닉스 세계 매출 자리 빅테크 수

 2. 2025-06-19  엔비디아:0 / 삼성:0 / 반도체:4
    텍사스인스트루먼트, 美서 반도체 생산 확대…82조원 투자
    🔗 http://www.edaily.co.kr/news/newspath.asp?newsid=02728966642203360
    highlight: ..<b>반도체</b> 기업 텍사스인스트루먼트(TI.. 이는 <b>엔비디아</b>·AMD 등 인공지능(AI) 칩과 달리, 자동차·의료기기·가전 등 실생활 전반에 쓰이는 핵심 부품이다.....<b>반도체</b>가...
    content  : [이데일리 방성훈 기자] 미국 반도체 기업 텍사스인스트루먼트(TI)가 미국 내 반도체 생산을 확대하기 위해 최소 600억달러(약 82조 6000억원)를 투자하기로 했다. (사진=AFP) 18일(현지시간) CNN
    t

# 2. 뉴스 검색 api

In [3]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
API_KEY

# 2. 요청 URL
url = "https://tools.kinds.or.kr/search/news"

# 3. 요청 페이로드
search_conditions = {
    "news_ids": [],
    "query": "fomc",   
    "from": "2024-06-01", 
    "until": "2025-06-22",
    "provider": ["서울경제"],           
    "category": [],             
    "fields": ["title", "news_id","published_at", "content", "provider", "byline",
               "provider_link_page","dateline","enveloped_at","hilight"]  

            #    tms_raw_stream
}

# 4. 요청 payload 구성
payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": search_conditions["news_ids"],
        # "news_ids"
        # "news_ids": [
            # "20250408000000.KIN00000000000000",
            # "20250408000000.KIN00000000000001",
            # "20250408000000.KIN00000000000002"
        # ],
        "query": search_conditions["query"],
        "published_at": {
            "from": search_conditions["from"],
            "until": search_conditions["until"]
        },
        "provider": search_conditions["provider"],
        "category": search_conditions["category"],
        "category_incident": [],
        "provider_subject": [],
        "subject_info": [],
        "sort": {"date": "desc"},
        "return_from": 0,
        "return_size": 200,
        "fields": search_conditions["fields"]
    }
}

# 5. 요청 보내기
response = requests.post(url, json=payload)

if response.status_code == 200:
    print("성공")
    result = response.json()
    documents = result["return_object"]["documents"]
    total_hits = result["return_object"]["total_hits"]
documents

성공


[{'news_id': '02100311.20250621070136001',
  'title': '강경한 이란, 미·일 무역협상 난항…꼬이는 국제정세에 시장 횡보[데일리국제금융시장]',
  'content': '지정학적 갈등과 경제 불확실성 등 주요 불안 요인이 계속되면서 뉴욕증시는 혼조 마감했다. 7월 금리 인하 기대로 상승 출발했던 증시는 이란이 우라늄 농축을 포기하지 않겠다는 의사를 유럽에 밝힌 것으로 전해지면서 흔들렸다. 미국이 삼성전자 등 해외기업이 중국에서 운영하는 생산시설에서 미국산 반도체 장비를 도입할 때 일일이 검토하겠다는 소식도 미·중 갈등의 우',
  'published_at': '2025-06-21T00:00:00.000+09:00',
  'enveloped_at': '2025-06-21T07:01:36.000+09:00',
  'dateline': '2025-06-21T06:59:03.000+09:00',
  'provider': '서울경제',
  'byline': '뉴욕=김흥록 특파원',
  'provider_link_page': 'http://www.sedaily.com/NewsView/2GU5QF7P8Z'},
 {'news_id': '02100311.20250621012531001',
  'title': '월러 이사 “7월 금리 인하 가능”…연준, 정책 이견 본격화',
  'content': '크리스토퍼 월러 연방준비제도(Fed·연준) 이사가 7월 기준금리 인하를 지지하고 나섰다. 관세와 이민 억제 등 도널드 트럼프 행정부의 정책에 따른 경제 불확실성 속에서 통화 정책의 방향을 둘러싼 연준 내부의 이견이 본격화하고 있다. \n \n\n 월러 이사는 20일(현지 시간) CNBC와의 인터뷰에서 “개인적인 의견이고 다른 위원들이 동의할지는 모르겠지만 우리(',
  'published_at': '2025-06-21T00:00:00.000+09:00',
  'enveloped_at': '2025-06-21T01:25:31.000+09:00'

# 3. 뉴스 조회 api

In [ ]:
docs = result["return_object"]["documents"]

for i, doc in enumerate(docs[:5], 1):
    print(f"{i}. 뉴스 ID:", doc["news_id"])


In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 검색 파라미터 정의
query = ""
from_date = "2025-05-10"
until_date = "2025-05-10"
provider = ["서울경제"]
preview_length = 200  # 기사 본문 미리보기 길이

# 3. 뉴스 검색 API 요청
search_url = "https://tools.kinds.or.kr/search/news"

search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "provider": provider,
        "category": [],
        "category_incident": [],
        "provider_subject": [],
        "subject_info": [],
        "sort": { "date": "desc" },
        "return_from": 0,
        "return_size": 5,
        "fields": ["news_id", "title", "published_at"]
    }
}

search_response = requests.post(search_url, json=search_payload)

if search_response.status_code == 200:
    search_result = search_response.json()
    docs = search_result["return_object"]["documents"]
    total_hits = search_result["return_object"]["total_hits"]

    if not docs:
        print("❗ 뉴스 검색 결과가 없습니다.")
    else:
        print("✅ 뉴스 검색 API 호출 성공!")
        print(f"📦 총 검색 결과: {total_hits}건\n")

        # 뉴스 ID 추출
        news_ids = [doc["news_id"] for doc in docs]
        for i, doc in enumerate(docs, 1):
            print(f"{i}. 뉴스 ID: {doc['news_id']} | 제목: {doc['title']}")
        
        print("\n📥 뉴스 ID로 상세 기사 조회 시작...\n")

        # 4. 뉴스 조회 API 요청
        view_payload = {
            "access_key": API_KEY,
            "argument": {
                "news_ids": news_ids,
                "fields": [
                    "title",
                    "content",
                    "byline",
                    "category",
                    "published_at",
                    "provider",
                    "images",
                    "images_caption"
                ]
            }
        }

        view_response = requests.post(search_url, json=view_payload)

        if view_response.status_code == 200:
            view_result = view_response.json()
            print("✅ 뉴스 조회 API 호출 성공!\n")

            for i, doc in enumerate(view_result["return_object"]["documents"], 1):
                print(f"----- {i}번째 뉴스 -----")
                print("제목:", doc.get("title", "없음"))
                print("기자:", doc.get("byline", "없음"))
                print("발행일:", doc.get("published_at", "없음"))
                print("카테고리:", doc.get("category", []))
                print("언론사:", doc.get("provider", "없음"))
                print("본문 (앞부분):", doc.get("content", "")[:preview_length], "...")

                # 이미지 있을 경우만 출력
                image_url = doc.get("images")
                if image_url:
                    print("🖼 이미지 URL:", image_url)
                    print("📝 이미지 설명:", doc.get("images_caption", "설명 없음"))
                print()

        else:
            print("❌ 뉴스 조회 API 오류:", view_response.status_code)
            try:
                print(view_response.json().get("reason", "알 수 없는 오류"))
            except:
                print(view_response.text)

else:
    print("❌ 뉴스 검색 API 오류:", search_response.status_code)
    try:
        print(search_response.json().get("reason", "알 수 없는 오류"))
    except:
        print(search_response.text)


# 4. 오늘의 이슈 api(이슈 랭킹 api)

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. 환경 변수에서 API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 날짜 & 언론사 설정
url = "https://tools.kinds.or.kr/issue_ranking"
target_date = "2025-06-07"
provider = [""]  # 생략 가능

# 3. 요청 바디 구성
payload = {
    "access_key": API_KEY,
    "argument": {
        "date": target_date,
        "provider": provider
    }
}

# 4. API 호출
response = requests.post(url, json=payload)

# 5. 결과 처리
if response.status_code == 200:
    result = response.json()
    topics = result["return_object"].get("topics", [])

    print(f"\n✅ 이슈 랭킹 API 호출 성공! ({target_date})\n")

    if not topics:
        print("📭 이 날은 이슈가 없어요. 조용한 날인가봐요.")
    else:
        # topic_rank 기준으로 오름차순 정렬 (중요도 높은 순)
        topics = sorted(topics, key=lambda x: x["topic_rank"])

        for i, topic in enumerate(topics, 1):
            print(f"🧠 {i}. 이슈 제목: {topic['topic']}")
            print(f"   🔗 주요 키워드: {topic['topic_keyword']}")
            print(f"   🔢 중요도 (topic_rank): {topic['topic_rank']}")
            print(f"   📰 관련 기사 수: {len(topic['news_cluster'])}")
            
            # 뉴스 ID 리스트 (선택 출력)
            print("   🆔 뉴스 ID:")
            for news_id in topic["news_cluster"]:
                print(f"     - {news_id}")
            print("-" * 80)

else:
    print("❌ 이슈 랭킹 API 오류 발생:", response.status_code)
    try:
        print("사유:", response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


In [ ]:
topic

In [ ]:
topic

In [ ]:
topic

# 5. 연관어 분석 api(워드 클라우드)

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. 환경 변수에서 API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 엔드포인트
url = "https://tools.kinds.or.kr/word_cloud"

# 3. 요청 페이로드 구성
payload = {
    "access_key": API_KEY,
    "argument": {
        "query": "삼성",
        "published_at": {
            "from": "2025-06-07",
            "until": "2025-06-10"
        },
        "provider": ["02100311"],
        "category": [],
        "category_incident": [],
        "byline": "",
        "provider_subject": [],
        "sort": "accuracy",
        "return_size": 100
    }
}

# 4. 헤더 설정
headers = {
    "Content-Type": "application/json"
}

# 5. API 호출
response = requests.post(url, json=payload, headers=headers)

# 6. 결과 처리
if response.status_code == 200:
    nodes = response.json()["return_object"].get("nodes", [])
    if not nodes:
        print("연관 키워드가 없습니다.")
    else:
        print(f"상위 {len(nodes)}개 연관 키워드 (정확도순):")
        for i, node in enumerate(nodes, 1):
            print(f"{i}. {node['name']:<15} (weight: {node['weight']:.3f})")
else:
    err = response.json().get("reason") or response.json().get("message")
    print("연관어 분석 API 오류 발생:", response.status_code, err or response.text)


# 6. 키워드 트렌드 api(뉴스 타임라인)

In [52]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 1. 맥북 기본 한글 폰트 지정 (AppleGothic 또는 NanumGothic)
plt.rcParams['font.family'] = 'AppleGothic'

# 2. 마이너한 깨짐 방지 (음수 표시 에러 방지)
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt

# 1. API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 요청 설정
url = "https://tools.kinds.or.kr/time_line"
query_keyword = "인공지능"
from_date = "2025-03-01"
until_date = "2025-04-14"
provider = ["서울경제"]
interval = "day"  # day, month, year

# 3. 요청 바디 구성
payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query_keyword,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "provider": provider,
        "category": [],
        "category_incident": [],
        "byline": "",
        "provider_subject": [],
        "interval": interval,
        "normalize": "false"
    }
}

# 4. API 호출
response = requests.post(url, json=payload)

# 5. 결과 처리
if response.status_code == 200:
    result = response.json()
    timeline = result["return_object"].get("time_line", [])

    print(f"\n✅ 키워드 트렌드 API 호출 성공! (키워드: '{query_keyword}')\n")
    
    if not timeline:
        print("📭 기간 내 데이터 없음.")
    else:
        print("📊 날짜별 키워드 언급량:\n")
        dates = []
        counts = []
        for point in timeline:
            label = point["label"]
            count = point["hits"]
            print(f"{label} → {count}건")
            dates.append(label)
            counts.append(count)

        # ⬇ 시각화 (옵션)
        df = pd.DataFrame({"날짜": dates, "기사 수": counts})
        df["날짜"] = pd.to_datetime(df["날짜"], format="%Y%m%d")
        df.set_index("날짜", inplace=True)

        df.plot(kind="line", figsize=(12, 5), marker="o", title=f"'{query_keyword}' 키워드 트렌드")
        plt.ylabel("기사 수")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

else:
    print("❌ 키워드 트렌드 API 오류:", response.status_code)
    try:
        print(response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


# 7. 인기검색어 api

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 요청 설정
url = "https://tools.kinds.or.kr/query_rank"

from_date = "2025-04-01"
until_date = "2025-04-14"
offset = 10  # 가져올 인기 키워드 수

payload = {
    "access_key": API_KEY,
    "argument": {
        "from": from_date,
        "until": until_date,
        "offset": offset
        # "target_access_key": ""  ← 특정 기관 사용자 한정할 경우만 사용
    }
}

# 3. API 호출
response = requests.post(url, json=payload)

# 4. 결과 처리
if response.status_code == 200:
    result = response.json()
    keywords = result["return_object"].get("queries", [])

    print(f"✅ 인기검색어 API 호출 성공! ({from_date} ~ {until_date})\n")

    if not keywords:
        print("📭 인기 검색어가 없습니다.")
    else:
        print("📌 인기 검색어 목록:\n")
        for i, item in enumerate(keywords, 1):
            print(f"{i}. 🔍 키워드: {item['query']} | 🔢 검색 횟수: {item['count']}")
else:
    print("❌ 인기 검색어 API 오류 발생:", response.status_code)
    try:
        print("사유:", response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


# 8. 뉴스 인용문 검색 api

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 요청 URL
url = "https://tools.kinds.or.kr/search/quotation"

# 3. 요청 조건 설정
query = "삼성"
from_date = "2025-06-08"
until_date = "2025-06-09"
provider = ["서울경제"]

# 4. 요청 바디 구성
payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "provider": provider,
        "category": [],
        "category_incident": [],
        "byline": "",
        "provider_subject": [],
        "subject_info": [],
        "sort": { "date": "desc" },
        "hilight": 200,
        "return_from": 0,
        "return_size": 10,
        "fields": [
            "quotation",
            "provider",
            "byline",
            "published_at",
            "title"
        ]
    }
}

# 5. API 호출
response = requests.post(url, json=payload)

# 6. 결과 처리
if response.status_code == 200:
    result = response.json()
    docs = result["return_object"]["documents"]
    print(f"\n✅ 뉴스 인용문 API 호출 성공! (총 {len(docs)}건)\n")

    if not docs:
        print("📭 인용문 데이터 없음")
    else:
        for i, doc in enumerate(docs, 1):
            print(f"----- {i} -----")
            print("📰 제목:", doc.get("title", "없음"))
            print("🧾 발언:", doc.get("quotation", "❌ 없음"))
            print("👤 기자:", doc.get("byline", "없음"))
            print("🗓 발행일:", doc.get("published_at", "없음"))
            print("📰 언론사:", doc.get("provider", "없음"))
            print()
else:
    print("❌ 뉴스 인용문 API 오류 발생:", response.status_code)
    try:
        print(response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


# 9. 오늘의 키워드 api(분야별 키워드))

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
url = "https://tools.kinds.or.kr/today_category_keyword"

# 2. 요청 바디 구성
payload = {
    "access_key": API_KEY,
    "argument": {}
}

# 3. API 호출
response = requests.post(url, json=payload)

# 4. 결과 처리
if response.status_code == 200:
    result = response.json()
    print("✅ 오늘의 키워드 API 호출 성공!\n")

    print("📊 분야별 기사 비중 (cate_ratio):\n")
    for item in result["return_object"].get("cate_ratio", []):
        print(f"🗂 분야: {item['category_name']}")
        print(f"   📈 비중: {item['category_percent']}%")
        print(f"   📰 기사 수: {item['category_count']}건\n")

    print("🔑 분야별 주요 키워드 (cate_keyword):\n")
    for item in result["return_object"].get("cate_keyword", []):
        print(f"📌 분야: {item['category_name']}")
        print(f"   🔍 키워드: {item['named_entity']}")
        print(f"   📊 등장 횟수: {item['named_entity_count']}")
        print(f"   🧠 타입: {item['named_entity_type']} | 단계: {item['entity_step']}")
        print("-" * 50)

else:
    print("❌ 오늘의 키워드 API 오류:", response.status_code)
    try:
        print("사유:", response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


# 10. 특성 추출 api

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. 환경 변수 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. 요청 URL
url = "https://tools.kinds.or.kr/feature"

# 3. 분석할 기사 텍스트 입력
payload = {
    "access_key": API_KEY,
    "argument": {
        "title": "AI 채용 시대… 서울시, 비대면 면접 체험 확대",
        "sub_title": "AI 면접 역량 검사 프로그램, 청년 구직자 대상 운영",
        "content": """
        서울시는 청년들이 급변하는 취업 트렌드에 발맞춰 비대면 AI 채용에 대비할 수 있도록
        AI 면접 체험과 역량 검사 프로그램을 제공한다.
        이 프로그램은 지난해 1만 5506명의 청년 구직자가 이용했으며 95.8%의 만족도를 보였다.
        """
    }
}

# 4. API 호출
response = requests.post(url, json=payload)

# 5. 결과 처리
if response.status_code == 200:
    result = response.json()
    print("✅ 특성 추출 API 호출 성공!\n")

    for field in ["title", "sub_title", "content"]:
        print(f"📌 {field.upper()} 키워드:")
        text = result["return_object"]["result"].get(field, "")
        
        if not text.strip():
            print(" - (데이터 없음)")
            continue
        
        for item in text.strip().split(" "):  # 공백으로 나눔
            if "|" in item:
                keyword, score = item.split("|")
                print(f" - 🔑 {keyword} (점수: {score})")
            else:
                print(f" - 🔑 {item} (점수 없음)")
        print()

else:
    print("❌ 특성 추출 API 오류:", response.status_code)
    try:
        print(response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


# 11. 키워드 추출 api

In [ ]:
import requests
import os
from dotenv import load_dotenv

# 1. 환경 변수 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

# 2. API URL
url = "https://tools.kinds.or.kr/keyword"

# 3. 본문 입력
payload = {
    "access_key": API_KEY,
    "argument": {
        "title": "AI 채용 시대… 서울시, 비대면 면접 체험 확대",
        "sub_title": "AI 면접 역량 검사 프로그램, 청년 구직자 대상 운영",
        "content": """
        서울시는 청년들이 급변하는 취업 트렌드에 발맞춰 비대면 AI 채용에 대비할 수 있도록
        AI 면접 체험과 역량 검사 프로그램을 제공한다.
        """
    }
}

# 4. API 호출
response = requests.post(url, json=payload)

# 5. 결과 출력
if response.status_code == 200:
    result = response.json()
    print("✅ 키워드 추출 API 호출 성공!\n")

    for field in ["title", "sub_title", "content"]:
        keywords = result["return_object"]["result"].get(field, "")
        print(f"📌 {field.upper()} 키워드:")
        for word in keywords.strip().split(" "):
            if word:
                print(f" - 🔑 {word}")
        print()
else:
    print("❌ 키워드 추출 API 오류:", response.status_code)
    try:
        print(response.json().get("reason", "알 수 없는 오류"))
    except:
        print(response.text)


# 12. topn 키워드 api

In [ ]:
import requests
import os
import sys
from dotenv import load_dotenv

def main():
    # 1. API 키 로딩 및 검증
    load_dotenv()
    api_key = os.getenv("BIGKINDS_KEY")
    if not api_key:
        print("Error: BIGKINDS_KEY 환경 변수가 설정되지 않았습니다.", file=sys.stderr)
        sys.exit(1)

    # 2. 엔드포인트 및 헤더
    url = "https://tools.kinds.or.kr/topn_keyword"
    headers = {
        "Content-Type": "application/json"
    }

    # 3. 요청 페이로드 구성
    payload = {
        "access_key": api_key,
        "argument": {
            "published_at": {
                "from": "2025-06-07",
                "until": "2025-06-10"   # 종료일 포함하려면 하루를 더하세요
            },
            "query": "삼성"
            # 필요 시 다음 필드 추가
            # "provider": ["02100311"],
            # "return_size": 50
        }
    }

    # 4. API 호출
    try:
        resp = requests.post(url, json=payload, headers=headers, timeout=10)
    except requests.RequestException as e:
        print(f"Error: API 호출 중 예외 발생: {e}", file=sys.stderr)
        sys.exit(1)

    # 5. HTTP 상태 확인
    if resp.status_code != 200:
        print(f"Error: HTTP {resp.status_code} 반환", file=sys.stderr)
        sys.exit(1)

    # 6. 결과 파싱
    try:
        data = resp.json()
    except ValueError:
        print("Error: 응답을 JSON으로 파싱하지 못했습니다.", file=sys.stderr)
        sys.exit(1)

    # 7. API 내부 오류 체크
    if data.get("result") != 0:
        reason = data.get("reason") or data.get("message") or "알 수 없는 오류"
        print(f"Error: API 내부 실패 – {reason}", file=sys.stderr)
        sys.exit(1)

    # 8. 키워드 목록 추출 및 출력
    keywords = data.get("return_object", {}).get("result", [])
    if not keywords:
        print("추출된 키워드가 없습니다.")
    else:
        print("TopN 키워드 목록:")
        for item in keywords:
            rank = item.get("rank")
            name = item.get("name")
            print(f"{rank}. {name}")

if __name__ == "__main__":
    main()


# 빅카인즈 예제